# Eigenmodes, steady-state problems and efficient time integration

#### Job Marcelis, Ernani Hazbolatow, Koen Verlaan

In [1]:
from eigenmodes import *

## 1: Eigenmodes of drums or membranes of different shapes

To discretize $\nabla^2v(x, y) = Kv(x, y)$, we use a 5-point stencil, namely: $\frac{1}{h^2}(v_{i+1, j} + v_{i-1, j} + v_{i, j+1} + v_{i, j-1} -4v_{i,j})$. As an example, we use the following 4 by 4 system:

In [ ]:
draw_system_and_matrix(N=4, h=1, plot_system=True)

Since this is a 4 by 4 system, we have 16 gridpoints, meaning that our resulting matrix is 16 x 16. The entries in the matrix must placed in such a way that it corresponds to the 5-point stencil mentioned earlier:

In [ ]:
draw_system_and_matrix(N=4, h=1, print_latex_matrix=True, plot_system=False)

This matrix looks like:

$$
\left[\begin{array}{cccccccccccccccc}-4.0 & 1.0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\1.0 & -4.0 & 1.0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 1.0 & -4.0 & 1.0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 1.0 & -4.0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\1.0 & 0 & 0 & 0 & -4.0 & 1.0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 1.0 & 0 & 0 & 1.0 & -4.0 & 1.0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 1.0 & 0 & 0 & 1.0 & -4.0 & 1.0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 1.0 & 0 & 0 & 1.0 & -4.0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 0 & -4.0 & 1.0 & 0 & 0 & 1.0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 1.0 & -4.0 & 1.0 & 0 & 0 & 1.0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 1.0 & -4.0 & 1.0 & 0 & 0 & 1.0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 1.0 & -4.0 & 0 & 0 & 0 & 1.0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 0 & -4.0 & 1.0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 1.0 & -4.0 & 1.0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 1.0 & -4.0 & 1.0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1.0 & 0 & 0 & 1.0 & -4.0\end{array}\right]
$$

We have implemented 3 different shapes for the drums/membranes. For each shape the matrix is constructed slightly differently and the equation $Mv = Kv$ is solved using `scipy.sprase.linalg.eigs`. This was done because the matrices are sparse by nature and the calculation would therefore be slow when using `scipy` functions designed for dense matrices. Below we plot the eigenmodes for some of the smallest eigenfrequencies for each shape. The parameter `freqs_of_interest` lets the user decide which modes are shown. Modes 1 (index 0) - 10 (index 9) are calculated. 

Square:

In [ ]:
L = 1
h = 0.01
N = int(L/h) - 1
M_square = construct_M_square(N, h)
lamda, v = linalg.eigs(M_square, k=4, which='SM')
plot_eigenvectors(lamda, v, grid_shape=(N,N), L=L, freqs_of_interest=[0, 1, 2, 3])

Rectangle:

In [ ]:
L = 1
hx = 0.01
hy = 0.02
Nx = int(L/hx) - 1
Ny = int(2*L/hy) - 1
M_rec = construct_M_rec(Nx, Ny, hx, hy)
lamda, v = linalg.eigs(M_rec, k=4, which='SM')
plot_eigenvectors(lamda, v, grid_shape=(Ny, Nx), L=L, shape='rectangle')

Circle:

In [ ]:
M_circle = construct_M_circle(N, h, L)
lamda, v = linalg.eigs(M_circle, k=4, which='SM')
plot_eigenvectors(lamda, v, grid_shape=(N,N), L=L, freqs_of_interest=[0, 1, 2, 3])

As mentioned above, we use `eigs()` to compute the eigenvalues and eigenvector since the matrix is sparse. To obtain insight in how much faster `eigs()` is compared to `eig()`, we test the performance on a square grid with different system sizes. Each system size is done 25 times to obtain an average with confidence intervals.

In [ ]:
Ns = np.linspace(10, 30, 3)
num_runs = 25
mean_eig, CI_eig, mean_eigs, CI_eigs = performance_compare(Ns, num_runs, plot=True)